<a href="https://colab.research.google.com/github/FlierK254/Dholuo-Swahili-Neural-Machine-Translator/blob/main/Dholuo_Swahili_Machine_Translator_using_OpenNMT_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mounting the Drive**

In [ ]:
import os
os.listdir()
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive



**Installing OpenMT -py**



In [ ]:
# Install OpenNMT-py

!pip3 install OpenNMT-py

**Checking if GPU is available**

In [ ]:
import torch

torch.cuda.is_available()
torch.cuda.get_device_name(0)

'Tesla T4'

**Listing available files**

In [ ]:
!ls /content/drive/MyDrive/OpenMT/


Brn_O_Test.txt						  N_combined_O_train.txt  pred_1000.txt
C_Test_O.txt						  N_combined_O_val.txt	  pred_Brn_test.txt
Dholuo_Swahili_Machine_Translator_using_OpenNMT_py.ipynb  N_combined_T_train.txt  pred_Test2.txt
N_combined_O_test.txt					  N_combined_T_val.txt	  run


**Displaying the first 10 line of the Source file**

In [ ]:

!head -n 10 /content/drive/MyDrive/OpenMT/N_combined_O_train.txt


6am dala fm news 9th march 2021 headline.
migawo ma ochung ne weche medo remo, kenya national blood transfusion services otudore gi migawo mar coalition of blood for africacoba echenro mar neno ni jopiny ogolo remo maromo units gana 3 ei ndalo 3 manyalo konyo mine mawito remo mathoth kinde magikonyore.
chienrono ma otisi ni keep mothers alive ibiro itayo egwenge 3 epachoka kaka yor hingo odiochieng mine ebuo piny mangima ka ochopo egikone kawuono.
ondamono okaw bang yangore ni atamalo 34 mag mine wito ngimagi epachoka kaluwore giremo machuer mangeny kinde magikonyore.
dr. nduku kilonzo en jatend migawo ma ochung ne golo kod medo remo epachoka.
kaluwore gi jataa tije e migawo mar coalition of blood for africacoba migosi richard kiplagat, atamalo 60 mar remo majopiny golo epachoka duarore mondo ores ngima mine ma konyore.
okuayo jopiny obed kagolo remo mondo ores ngima jowadgi.
jalup ker migosi william ruto okwedo jogo malero ni othuwo jopiny kotenore gi wach mar hustler ma otiyogo.
migo

**Checking the number of lines in the Source file**

In [ ]:
#Check the number of lines in the source file

!echo "Number of lines:" && wc -l /content/drive/MyDrive/OpenMT/N_combined_O_train.txt


Number of lines:
3027 /content/drive/MyDrive/OpenMT/N_combined_O_train.txt


**Creating a Configurations file**

In [ ]:
# Create the YAML configuration file

config = '''# dholuo_swahili.yaml

## Where the samples will be written
save_data: /content/drive/MyDrive/OpenMT/run
## Where the vocab(s) will be written
src_vocab: /content/drive/MyDrive/OpenMT/run/vocab.src
tgt_vocab: /content/drive/MyDrive/OpenMT/run/vocab.tgt

## Where the model will be saved
save_model: /content/drive/MyDrive/OpenMT/model
# Prevent overwriting existing files in the folder
overwrite: True
# Corpus opts:
data:
    corpus_1:
        path_src: /content/drive/MyDrive/OpenMT/N_combined_O_train.txt
        path_tgt: /content/drive/MyDrive/OpenMT/N_combined_T_train.txt
    valid:
        path_src: /content/drive/MyDrive/OpenMT/N_combined_O_val.txt
        path_tgt: /content/drive/MyDrive/OpenMT/N_combined_T_val.txt

# Train on a single GPU
world_size: 1
gpu_ranks: [0]
save_checkpoint_steps: 500
train_steps: 1000
valid_steps: 500
early_stopping: 5
'''

with open("dholuo_swahili.yaml", "w+") as config_yaml:
  config_yaml.write(config)

!cat dholuo_swahili.yaml

# dholuo_swahili.yaml

## Where the samples will be written
save_data: /content/drive/MyDrive/OpenMT/run


## Where the vocab(s) will be written
src_vocab: /content/drive/MyDrive/OpenMT/run/vocab.src
tgt_vocab: /content/drive/MyDrive/OpenMT/run/vocab.tgt

## Where the model will be saved
save_model: /content/drive/MyDrive/OpenMT/model

# Prevent overwriting existing files in the folder
overwrite: True

# Corpus opts:
data:
    corpus_1:
        path_src: /content/drive/MyDrive/OpenMT/N_combined_O_train.txt
        path_tgt: /content/drive/MyDrive/OpenMT/N_combined_T_train.txt
    valid:
        path_src: /content/drive/MyDrive/OpenMT/N_combined_O_val.txt
        path_tgt: /content/drive/MyDrive/OpenMT/N_combined_T_val.txt

# Train on a single GPU
world_size: 1
gpu_ranks: [0]
save_checkpoint_steps: 500
train_steps: 1000
valid_steps: 500
early_stopping: 5


**Building the Vocabularly**

In [ ]:
# Build Vocabulary
!onmt_build_vocab -config dholuo_swahili.yaml -n_sample -1

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-06-18 07:45:06,564 INFO] Counter vocab from -1 samples.
[2024-06-18 07:45:06,564 INFO] n_sample=-1: Build vocab on full datasets.
[2024-06-18 07:45:07,045 INFO] Counters src: 8835
[2024-06-18 07:45:07,046 INFO] Counters tgt: 9623
[2024-06-18 07:45:07,046 WARNING] path /content/drive/MyDrive/OpenMT/run/vocab.src exists, may overwrite...
[2024-06-18 07:45:07,075 WARNING] path /content/drive/MyDrive/OpenMT/run/vocab.tgt exists, may overwrite...


**Checking if the GPU is active and visible to PyTorch**

In [ ]:
# Check if GPU is active
# If not, go to "Runtime" menu > "Change runtime type" > "GPU"
!nvidia-smi -L

# Make sure the GPU is visible to PyTorch

import torch

gpu_id = torch.cuda.current_device()
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(gpu_id))

GPU 0: Tesla T4 (UUID: GPU-d2dde467-a678-75d2-3c7f-be10e4b9fa3f)
True
Tesla T4


**Train the Neural Machine Translator model**

In [ ]:
# Train the Neural Machine Translator (NMT)
!onmt_train -config dholuo_swahili.yaml

Streaming output truncated to the last 5000 lines.
			* corpus_1: 2376
[2024-06-18 07:57:10,729 INFO] Weighted corpora loaded so far:
			* corpus_1: 2410
[2024-06-18 07:57:10,767 INFO] Weighted corpora loaded so far:
			* corpus_1: 2377
[2024-06-18 07:57:10,802 INFO] Weighted corpora loaded so far:
			* corpus_1: 2411
[2024-06-18 07:57:10,847 INFO] Weighted corpora loaded so far:
			* corpus_1: 2378
[2024-06-18 07:57:10,880 INFO] Weighted corpora loaded so far:
			* corpus_1: 2412
[2024-06-18 07:57:10,918 INFO] Weighted corpora loaded so far:
			* corpus_1: 2379
[2024-06-18 07:57:10,960 INFO] Weighted corpora loaded so far:
			* corpus_1: 2413
[2024-06-18 07:57:10,988 INFO] Weighted corpora loaded so far:
			* corpus_1: 2380
[2024-06-18 07:57:11,033 INFO] Weighted corpora loaded so far:
			* corpus_1: 2414
[2024-06-18 07:57:11,059 INFO] Weighted corpora loaded so far:
			* corpus_1: 2381
[2024-06-18 07:57:11,121 INFO] Weighted corpora loaded so far:
			* corpus_1: 2415
[2024-06-18 07:5

**Display the first 10 rows of the test file**

In [ ]:
#List the test file
!head -n 10 /content/drive/MyDrive/OpenMT/N_combined_O_test.txt

jopiny ebomani ma kisumu okuayo ne moluor odinga chang mapiyo.
jopiny onyis obi oti gi thuolono echualo pachgi.
yuaruok omuoch eod buch county ma kakamega.
oruakgi ekar thieth ma othaya annex kama itemo rese ngimagi.
mach makakni owango mwandu mabeche thoth e kambi sukari ma mumias.
ojende ojiw odonj eweche bur.
dala fm news at 1pm on 25th march 2021.
chiemo jokenya million 1.4 onge kod nyalo mar yudo migago kata chiemo mapilepile.
oloso sama nyoro ji 25 ne otho kaluwore gi corona.
jamni kod gwen mane ni ei jikon owang duto.


In [ ]:
# Translate
!onmt_translate -model /content/drive/MyDrive/OpenMT/model_step_1000.pt -src /content/drive/MyDrive/OpenMT/N_combined_O_test.txt -output /content/drive/MyDrive/OpenMT/pred_Test2.txt -gpu 0 -verbose

[2024-06-18 09:17:18,261 INFO] Loading checkpoint from /content/drive/MyDrive/OpenMT/model_step_1000.pt
[2024-06-18 09:17:19,046 INFO] Loading data into the model
[2024-06-18 09:17:21,125 INFO] 
SENT 1: ['jopiny', 'ebomani', 'ma', 'kisumu', 'okuayo', 'ne', 'moluor', 'odinga', 'chang', 'mapiyo.']
PRED 1: serikali ya uchaguzi wa huko humu nchini. na mambo ya maendeleo nchini.
PRED SCORE: -1.2712

[2024-06-18 09:17:21,125 INFO] 
SENT 2: ['jopiny', 'onyis', 'obi', 'oti', 'gi', 'thuolono', 'echualo', 'pachgi.']
PRED 2: wananchi ya kipekee kwa uchaguzi wa maendeleo katika kaunti ya homabay.
PRED SCORE: -1.3302

[2024-06-18 09:17:21,126 INFO] 
SENT 3: ['yuaruok', 'omuoch', 'eod', 'buch', 'county', 'ma', 'kakamega.']
PRED 3: katika hospitali ya msingi ya oyola iliyoko iliyomo.
PRED SCORE: -1.1384

[2024-06-18 09:17:21,126 INFO] 
SENT 4: ['oruakgi', 'ekar', 'thieth', 'ma', 'othaya', 'annex', 'kama', 'itemo', 'rese', 'ngimagi.']
PRED 4: katika jiji la nairobi na shule ya msingi ya oyola iliyoko 

In [ ]:
# Download the multi-bleu.perl script
!wget https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/generic/multi-bleu.perl

# Make it executable
!chmod +x multi-bleu.perl

# Run the script with the reference and hypothesis files
!perl multi-bleu.perl /content/drive/MyDrive/OpenMT/N_combined_O_test.txt < /content/drive/MyDrive/OpenMT/pred_1000.txt
